In [72]:
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
from datetime import datetime
from re import search
import json
from time import sleep
from random import uniform

# Парсинг 
Вдохновившись новостями по нобелевской неделе, для парсинга я выбрал сайт Naked Science, который содержит научнопопулярные статьи и новости из мира науки. Из каждой статьи можно вытащить ее название, автора, аннотацию, индекс важности (оценка автором важности исследования/открытия для науки), дату публикации, сам текст статьи, хэштеги и количество просмотров в качестве целевой переменной.

In [7]:
def parse_date(date, form='%d.%m.%Y'):
    mapper = {
        'января': '01',
        'февраля': '02',
        'марта': '03',
        'апреля': '04',
        'мая': '05',
        'июня': '06',
        'июля': '07',
        'августа': '08',
        'сентября': '09',
        'октября': '10',
        'ноября': '11',
        'декабря': '12'
    }
    if search(r'\d{1,2}\.\d{2}\.\d{4}', date):
        return date
    else:
        date = date.split()
        day = '0' + date[0] if len(date[0]) < 2 else date[0]
        return '.'.join((day, mapper[date[1]], '2025'))

In [43]:
def get_info(obj):
    header = obj.find('a', attrs={'class': 'animate-custom'})
    link = header.attrs['href'].strip()
    
    title = header.get_text().split()
    imp_ind = float(title[-1])
    title = ' '.join(title[:-1])

    author = obj.find(lambda tag: tag.name == 'div' and tag.get('class') == ['meta-item', 'meta-item_author'])
    author = author.string.strip()

    date = obj.find('span', attrs={'class': 'echo_date'}).string
    date = parse_date(date.split(', ')[0])

    views = obj.find('span', attrs={'class': 'fvc-count'}).string
    views = int(views.replace(' ', ''))

    annot = obj.find('p').string.strip()

    tags = obj.find(lambda tag: tag.name == 'div' and tag.get('class') == ['terms-items', 'grid'])
    tags = tags.find_all('div', attrs={'class': 'terms-item'})
    tags = [t.get_text().replace('#', '').strip().lower() for t in tags]

    return {
        'title': title,
        'author': author,
        'date': date,
        'imp_ind': imp_ind,
        'views': views,
        'annot': annot,
        'tags': tags,
        'link': link
    }

In [79]:
user_ag = UserAgent()
main_link = 'https://naked-science.ru/article/page/'
pages = 10
articles = []
attemps = 0
success = []

for i in range(1, pages + 1):
    if i % 5 == 0:
        print(f'processing page {i}/{pages}')
        with open('articles_links.json', 'w', encoding='UTF-8') as f:
            json.dump(articles, f, ensure_ascii=False, indent=4)
    
    for j in range(3):
        sleep(uniform(1.1, 1.9))
        response = requests.get(main_link + f'{i}/', headers={'User-Agent': user_ag.random})
        if not response.ok:
            print(f'\n  WARNING: page {i} parsing failed\n')
            continue
        else:
            success.append(i)
            attemps = 0
            soup = BeautifulSoup(response.content, 'html.parser')
            news = soup.find('div', attrs={'class': 'news-items'})
            news = news.find_all(lambda tag: tag.name == 'div' and tag.get('class') == ['news-item-left', 'with-bookmark'])
            articles = articles + [get_info(x) for x in news if get_info(x) not in articles]
            break
    else:
        attemps += 1

    if attemps > 3:
        print(f'\n  WARNING: {i - 1} pages processed')
        break


print(f'\n --- {len(success)}/{pages} pages have been successfully parsed ---')

processing page 1/10
processing page 2/10
processing page 3/10
processing page 4/10
processing page 5/10
processing page 6/10
processing page 7/10
processing page 8/10
processing page 9/10
processing page 10/10

 --- 10/10 pages have been successfully parsed ---


In [80]:
len(articles)

198

In [86]:
smth = [i['title'] for i in articles]
len(smth), len(set(smth))

(198, 193)